# Define a model that serves as an example

### Export a PyTorch model to an ONNX model

In [10]:
import torchvision
import torch
import os
import time
from PIL import Image
import cv2
import numpy as np

print(torch.__version__)

onnx_model_path = 'resnet50.onnx'

input_name = ['input']
output_name = ['output']
input = torch.randn(1, 3, 224, 224).cuda()
model = torchvision.models.resnet50(pretrained=True).cuda()
torch.onnx.export(model, input, onnx_model_path, input_names=input_name, output_names=output_name, verbose=False)

1.5.0


### check onnx model

In [11]:
# import onnx
# onnx_model_path = 'resnet50.onnx'

# test = onnx.load(onnx_model_path)
# onnx.checker.check_model(test)
# print("==> Passed")

### convert an onnx model to a TensorRT

In [19]:
import tensorrt as trt

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

def get_engine(max_batch_size=1, onnx_file_path="", engine_file_path="", \
               fp16_mode=False, int8_mode=False, save_engine=True
               ):
    """Attempts to load a serialized engine if available, otherwise builds a new TensorRT engine and saves it."""

    def build_engine(max_batch_size, save_engine):
        """Takes an ONNX file and creates a TensorRT engine to run inference with"""
        with trt.Builder(TRT_LOGGER) as builder, \
            builder.create_network(EXPLICIT_BATCH) as network, \
            trt.OnnxParser(network, TRT_LOGGER) as parser:
            
            # 设置 builder 参数
            builder.max_workspace_size = 1 << 30  # Your workspace size
            builder.max_batch_size = max_batch_size
            builder.fp16_mode = fp16_mode  # Default: False
            builder.int8_mode = int8_mode  # Default: False
            if int8_mode:
                raise NotImplementedError

            # 解析模型
            if not os.path.exists(onnx_file_path):
                quit('ONNX file {} not found'.format(onnx_file_path))
            print('Loading ONNX file from path {}...'.format(onnx_file_path))
            with open(onnx_file_path, 'rb') as model:
                print('Beginning ONNX file parsing')
                parser.parse(model.read())
            print('Completed parsing of ONNX file')
            print('Building an engine from file {}; this may take a while...'.format(onnx_file_path))

            # 构建推理engine
            engine = builder.build_cuda_engine(network)
            print("Completed creating Engine")

            if save_engine:
                with open(engine_file_path, "wb") as f:
                    f.write(engine.serialize())
            return engine

    if os.path.exists(engine_file_path):
        # If a serialized engine exists, load it instead of building a new one.
        print("Reading engine from file {}".format(engine_file_path))
        with open(engine_file_path, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
            return runtime.deserialize_cuda_engine(f.read())
    else:
        return build_engine(max_batch_size, save_engine)

In [20]:
max_batch_size = 1
# These two modes are dependent on hardwares
fp16_mode = True
int8_mode = False
trt_engine_path = './model_fp16_{}_int8_{}.trt'.format(fp16_mode, int8_mode)
# Build an engine
engine = get_engine(max_batch_size, onnx_model_path, trt_engine_path, fp16_mode, int8_mode)

Loading ONNX file from path resnet50.onnx...
Beginning ONNX file parsing
Completed parsing of ONNX file
Building an engine from file resnet50.onnx; this may take a while...
Completed creating Engine


### Do inference with Pytorch & onnx

In [21]:
def get_img_np_nchw(filename):
    image = cv2.imread(filename)
    image_cv = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_cv = cv2.resize(image_cv, (224, 224))
    miu = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img_np = np.array(image_cv, dtype=float) / 255.
    r = (img_np[:, :, 0] - miu[0]) / std[0]
    g = (img_np[:, :, 1] - miu[1]) / std[1]
    b = (img_np[:, :, 2] - miu[2]) / std[2]
    img_np_t = np.array([r, g, b])
    img_np_nchw = np.expand_dims(img_np_t, axis=0)
    return img_np_nchw

filename = 'cat.jpeg'
img_np_nchw = get_img_np_nchw(filename)
img_np_nchw = img_np_nchw.astype(dtype=np.float32)

#### (1) Pytorch

In [22]:
model = torchvision.models.resnet50(pretrained=True).cuda()
resnet_model = model.eval()

for i in range(50):
    input_for_torch = torch.from_numpy(img_np_nchw).cuda()
    t3 = time.time()
    feat_2= resnet_model(input_for_torch)
    t4 = time.time()
    feat_2 = feat_2.cpu().data.numpy()
    print('Pytorch ok!')

    print("Inference time with the PyTorch model: {}".format(t4-t3))

Pytorch ok!
Inference time with the PyTorch model: 0.005634307861328125
Pytorch ok!
Inference time with the PyTorch model: 0.005437135696411133
Pytorch ok!
Inference time with the PyTorch model: 0.005404472351074219
Pytorch ok!
Inference time with the PyTorch model: 0.00537562370300293
Pytorch ok!
Inference time with the PyTorch model: 0.00644683837890625
Pytorch ok!
Inference time with the PyTorch model: 0.005829811096191406
Pytorch ok!
Inference time with the PyTorch model: 0.005459308624267578
Pytorch ok!
Inference time with the PyTorch model: 0.005388736724853516
Pytorch ok!
Inference time with the PyTorch model: 0.0053789615631103516
Pytorch ok!
Inference time with the PyTorch model: 0.005362272262573242
Pytorch ok!
Inference time with the PyTorch model: 0.005383014678955078
Pytorch ok!
Inference time with the PyTorch model: 0.005387067794799805
Pytorch ok!
Inference time with the PyTorch model: 0.005362033843994141
Pytorch ok!
Inference time with the PyTorch model: 0.005380630493

#### (2) TensorRT

In [23]:
import pycuda.driver as cuda
import pycuda.autoinit


class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        """Within this context, host_mom means the cpu memory and device means the GPU memory
        """
        self.host = host_mem
        self.device = device_mem

    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)

    def __repr__(self):
        return self.__str__()
    
def do_inference(context, bindings, inputs, outputs, stream, batch_size=1):
    # Transfer data from CPU to the GPU.
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # Run inference.
    context.execute_async(batch_size=batch_size, bindings=bindings, stream_handle=stream.handle)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
    # Return only the host outputs.
    return [out.host for out in outputs]


def postprocess_the_outputs(h_outputs, shape_of_output):
    h_outputs = h_outputs.reshape(*shape_of_output)
    return h_outputs

def allocate_buffers(engine):
    inputs = []
    outputs = []
    bindings = []
    stream = cuda.Stream()
    for binding in engine:
        size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
        dtype = trt.nptype(engine.get_binding_dtype(binding))
        # Allocate host and device buffers
        host_mem = cuda.pagelocked_empty(size, dtype)
        device_mem = cuda.mem_alloc(host_mem.nbytes)
        # Append the device buffer to device bindings.
        bindings.append(int(device_mem))
        # Append to the appropriate list.
        if engine.binding_is_input(binding):
            inputs.append(HostDeviceMem(host_mem, device_mem))
        else:
            outputs.append(HostDeviceMem(host_mem, device_mem))
    return inputs, outputs, bindings, stream

In [24]:
# Create the context for this engine
context = engine.create_execution_context()
# # Allocate buffers for input and output
inputs, outputs, bindings, stream = allocate_buffers(engine) # input, output: host # bindings

# Do inference
shape_of_output = (max_batch_size, 1000)
# Load data to the buffer
inputs[0].host = img_np_nchw.reshape(-1)

In [25]:
for i in range(50):
    t1 = time.time()
    trt_outputs = do_inference(context, bindings=bindings, inputs=inputs, outputs=outputs, stream=stream) # numpy data
    t2 = time.time()
    feat = postprocess_the_outputs(trt_outputs[0], shape_of_output)

    print('TensorRT ok')
    print("Inference time with the TensorRT engine: {}".format(t2-t1))
    
print('MSE Error = {}'.format(np.mean((feat - feat_2)**2)))

print('All completed!')

TensorRT ok
Inference time with the TensorRT engine: 0.0014615058898925781
TensorRT ok
Inference time with the TensorRT engine: 0.0012578964233398438
TensorRT ok
Inference time with the TensorRT engine: 0.0011913776397705078
TensorRT ok
Inference time with the TensorRT engine: 0.0011141300201416016
TensorRT ok
Inference time with the TensorRT engine: 0.001186370849609375
TensorRT ok
Inference time with the TensorRT engine: 0.0012629032135009766
TensorRT ok
Inference time with the TensorRT engine: 0.0011968612670898438
TensorRT ok
Inference time with the TensorRT engine: 0.0012068748474121094
TensorRT ok
Inference time with the TensorRT engine: 0.0021479129791259766
TensorRT ok
Inference time with the TensorRT engine: 0.0015397071838378906
TensorRT ok
Inference time with the TensorRT engine: 0.0012843608856201172
TensorRT ok
Inference time with the TensorRT engine: 0.00125885009765625
TensorRT ok
Inference time with the TensorRT engine: 0.001178741455078125
TensorRT ok
Inference time wi

### Experiments

##### Test Device: GTX 2080Ti        
##### Network:Resnet50                 
##### Inputsize:224x224

##### Pytorch Inference: 0.0053s

##### TensorRT with FP32: 0.0027s  

##### TensorRT with FP16: 0.0017s

##### TensorRT with FP16(change workspace from 1 >> 20 to 1>> 32): 0.0012s